In [1]:
import cv2
import tensorflow as tf 
import numpy as np

# defining class names

class_names = ['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street']

In [2]:
#importing model saved in tflite format

TF_MODEL_FILE_PATH = r"W:\tensorflow\models/intel_image_classification.tflite"

interpreter = tf.lite.Interpreter(TF_MODEL_FILE_PATH)

In [3]:
#getting input and output layers

interpreter.get_signature_list()

{'serving_default': {'inputs': ['rescaling_input'], 'outputs': ['dense_1']}}

In [4]:
# making a classifier for inferencing using the trained model  

classifier = interpreter.get_signature_runner()

In [5]:
# defining image path

image_path = r"W:\tensorflow\intel_image_classifcation\seg_pred\seg_pred\test.jpg"

#loading the image that's being predicted
pred_image = tf.keras.utils.load_img(image_path , target_size= (150, 150))

# converting image to float32 so the model can accept the image
pred_image = np.float32(pred_image)

# creating a batch
image_array = tf.expand_dims(pred_image, 0)

# scaling the logits returned by prediction into probabilities
predictions = classifier(rescaling_input= image_array)["dense_1"]
score = tf.nn.softmax(predictions)

# argmax returns the index
print(f"the image belongs to {class_names[np.argmax(score)]} with the confidence of {np.max(score)}")

the image belongs to mountain with the confidence of 0.9990416169166565


In [6]:
# defining the string that will be shown on the image. it includes the class name and the confidence is multiplied by 100 and only shown with upto two decimal places
result_string = f"{class_names[np.argmax(score)]} ({100 * np.max(score):.2f}%)"
print(result_string)

mountain (99.90%)


In [7]:
def visualization(image, result_string) -> np.ndarray:
    text_location = (30, 50)
    font_size = 4
    font_color = (0, 0, 255)
    font_thickness = 2
    cv2.putText(image, result_string, text_location, cv2.FONT_HERSHEY_PLAIN, font_size, font_color, font_thickness)
    
    return image

In [8]:
cv2_image = cv2.imread(image_path)

annotated_image = visualization(cv2_image, result_string)

cv2.imwrite(r"W:\tensorflow\intel_image_classifcation\cv2 inference\test.jpg", annotated_image)

# namedWindow fits a higher resolution image when using imshow
cv2.namedWindow("test", cv2.WINDOW_NORMAL)
cv2.imshow("test", annotated_image)
cv2.waitKey(0)
cv2.destroyAllWindows()